In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd
from tensorflow.contrib import rnn

In [2]:
data = pd.read_csv("/home/ace/vertika/test/train.csv")


train_x = data.values[:,1:]
train_y = data.values[:,0]
print(train_x)
print(train_y)
split_size = int(train_x.shape[0]*0.7)
print(split_size)

train_x, val_x = train_x[:split_size, : ], train_x[split_size: , :]
print(train_x.shape)
print(len(train_x))
print(val_x.shape)

train_y, val_y = train_y[:split_size], train_y[split_size:]
print(train_y.shape)
print(val_y.shape)

test_x = pd.read_csv( "/home/ace/vertika/test/test.csv"  ).values
test_x.shape

seed = 128
rng = np.random.RandomState(seed)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[1 0 1 ... 7 6 9]
29399
(29399, 784)
29399
(12601, 784)
(29399,)
(12601,)


In [3]:
time_steps=28
num_units=128        #hidden LSTM units
n_input=28           #rows of 28 pixels
learning_rate=0.001  #learning rate for adam
n_classes=10         #mnist is meant to be classified in 10 classes(0-9).
batch_size=256       #size of batch

epochs = 20

In [4]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

def preproc(unclean_batch_x):
    """Convert values to range 0-1"""
    temp_batch = unclean_batch_x / unclean_batch_x.max()
    
    return temp_batch

def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    if dataset_name == 'train':
        batch_x = train_x[batch_mask]
        batch_x = preproc(batch_x)
        batch_x = np.reshape(batch_x, ( batch_size, n_input, n_input))
        
        batch_y = train_y[batch_mask]
        batch_y = dense_to_one_hot(batch_y)
        return batch_x, batch_y
    elif dataset_name == 'val':
        batch_x = preproc(val_x)
        batch_x = np.reshape(batch_x, ( batch_size, n_input, n_input))
        
        batch_y = val_y
        batch_y = dense_to_one_hot(batch_y)
        return batch_x, batch_y
    else:
        batch_x = preproc(test_x)
        batch_x = np.reshape(batch_x, ( batch_size, n_input, n_input))
        return batch_x
    

In [5]:
tf.reset_default_graph()

out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

x=tf.placeholder("float",[None,time_steps,n_input])
print(x)
y=tf.placeholder("float",[None,n_classes])
print(y)

input=tf.unstack(x ,time_steps,1)
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#prediction=tf.matmul(outputs[-1],out_weights)+out_bias
logits = tf.matmul(outputs[-1],out_weights)+out_bias
prediction = tf.nn.softmax(logits)

Tensor("Placeholder:0", shape=(?, 28, 28), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)


In [6]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [7]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for ep in range(epochs):
        avg_cost = 0
        total_batch = len(train_x)/batch_size
        
        for bt in range(total_batch):
            batch_x, batch_y = batch_creator( batch_size, len(train_x), 'train')
            _, c = sess.run( [ opt, loss ] ,  feed_dict = { x:batch_x, y: batch_y } )
            avg_cost += c / total_batch
            
            if ep %5 == 0:
                if bt %100==0:
                    acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
                    los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})

                    #calculating test accuracy
                    batch_x, batch_y = batch_creator( len(val_x) , len(val_x), 'val')
                    print("Epoch:" + str(ep) + " Iter "+ str(bt) + "  Accuracy "+ str(acc) +  "  Loss "+ str(los)  + " Validation Accuracy:" + str(sess.run(accuracy, feed_dict={x:batch_x, y: batch_y})))

    
    
    batch_x = batch_creator( len(test_x) , len(test_x), 'test')
    print(len(batch_x))
    predict = tf.argmax(prediction, 1)
    print(predict)
    pred = predict.eval({x: batch_x})
    
    print("len of pred:" + str(len(pred)))
    
    outFile = open("/home/ace/vertika/test/out.csv", "w")
    outFile.write("ImageId,Label\n")
    for i in range( len(pred)):
        outFile.write( str(i+1) + "," + str(pred[i]) + "\n")
        
    print("File written\n")

Epoch:0 Iter 0  Accuracy 0.0625  Loss 2.680824 Validation Accuracy:0.09316721
Epoch:0 Iter 100  Accuracy 0.203125  Loss 2.107657 Validation Accuracy:0.22887072
Epoch:5 Iter 0  Accuracy 0.3359375  Loss 1.844715 Validation Accuracy:0.34671852
Epoch:5 Iter 100  Accuracy 0.3046875  Loss 1.8929927 Validation Accuracy:0.34814698
Epoch:10 Iter 0  Accuracy 0.38671875  Loss 1.7637151 Validation Accuracy:0.37481153
Epoch:10 Iter 100  Accuracy 0.37890625  Loss 1.668837 Validation Accuracy:0.37203395
Epoch:15 Iter 0  Accuracy 0.3828125  Loss 1.6853924 Validation Accuracy:0.3869534
Epoch:15 Iter 100  Accuracy 0.38671875  Loss 1.6745071 Validation Accuracy:0.3894929
28000
Tensor("ArgMax_2:0", shape=(?,), dtype=int64)
len of pred:28000
File written

